# cppyy tutorial
## all credit to Wim Lavrijsen for producing this nicely documented piece of software

In [1]:
from __future__ import print_function
p = print
import cppyy

## Define a class

In [2]:
cppyy.cppdef("""
  struct Integer {
    Integer(int i) : num(i) {}
    int num;
    private:
    int p_num = -1;
  };
""")

In [3]:
from cppyy.gbl import Integer
m1 = Integer(42)

p(m1.num)
try:
    p(m1.p_num)
except:
    p('p_num not available')

42
p_num not available


### Autogenerated help/doc

In [4]:
Integer?

In [5]:
help(Integer)

Help on class Integer:

class Integer(ObjectProxy)
 |  Method resolution order:
 |      Integer
 |      ObjectProxy
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __assign__(...)
 |      Integer& Integer::operator=(const Integer&)
 |      Integer& Integer::operator=(Integer&&)
 |  
 |  __init__(...)
 |      Integer::Integer(int i)
 |      Integer::Integer(const Integer&)
 |      Integer::Integer(Integer&&)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  num
 |      PyROOT property proxy (internal)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __cppname__ = 'Integer'
 |  
 |  __scope__ = ''
 |  
 |  ----------------------------------------------------------

## Dynamically add class methods

In [ ]:
cppyy.cppdef("""
    Integer add(Integer* m1, Integer* m2) {
        return Integer(m1->num + m2->num);
    }
""")
Integer.add = cppyy.gbl.add

In [ ]:
m2 = Integer(58)
p(m1.add(m2).num)

## Inheritance

In [ ]:
cppyy.cppdef("""
   struct Base {
       virtual std::string method() { return "Base"; }
   };
   
   struct Derived : Base {
       std::string method() override { return "Derived"; }
   };""")
Base, Derived = cppyy.gbl.Base, cppyy.gbl.Derived

In [ ]:
p(Base().method())
p(Derived().method())

## Operator overloading

In [ ]:
cppyy.cppdef("""
    struct IntValue {
        IntValue(int v) : num(v) {}
        IntValue &operator+(const IntValue &i) { num += i.num; return *this; }
        IntValue &operator+(int i) { num += i; return *this; }
        int num;
    };""")
IntValue = cppyy.gbl.IntValue

In [ ]:
IntValue.__repr__ = lambda self : str(self.num)

p(IntValue(1) + IntValue(1))
p(IntValue(3) + 7)
x, y, z = IntValue(1), IntValue(2), IntValue(3)
p(x + y + z)

## Template classes

In [6]:
vector = cppyy.gbl.vector
v = vector[int]((0, 8, 42)) # vector(int)(3, 42) also allowed
print(v[0], v[1], v[2])

AttributeError: Template instance has no attribute '__getitem__'

In [ ]:
cppyy.cppdef("""
template <typename T, int N>
struct fac
{
    static constexpr int value = N * fac<T, N-1>::value;
};

template <typename T>
struct fac<T, 0>
{
    static constexpr int value = 1;
};
""")
from cppyy.gbl import fac

In [ ]:
print(fac(int, 5).value)

## Function overloading

In [ ]:
cppyy.cppdef("""
double dub(double x) { return x*2; }

int dub(int x) { return x*2; }

template <typename T>
T dub(T t) { return t + t; }
""")
dub = cppyy.gbl.dub

In [ ]:
type(dub(3))

In [ ]:
type(dub(3.))

In [ ]:
i = IntValue(2)
i2 = dub(i)
print(i2, '---', type(i2))

## Template methods

In [ ]:
cppyy.cppdef("""
struct Doubler {
    int operator()(int t) { return t + t; } // at least one overload must be non-template

    template <typename T>
    T operator()(T t) { return t + t; }
};
""")
from cppyy.gbl import Doubler

In [ ]:
dub = Doubler()
six = dub(IntValue(3))
print(six, '---', type(six))

## Including a header

In [ ]:
f = open('/tmp/cppyy_test.h','w') 
f.write('std::string bar() { return "oh yeah"; }')
f.close() 
cppyy.include('/tmp/cppyy_test.h')

In [ ]:
p(cppyy.gbl.bar())

## Automatic creation of bindings with ROOT

[Bindings with ROOT](root_make_bindings.ipynb)

## TTree to numpy: pythonizations

In [27]:
%%bash
subl3 "/home/blue/ROOT/root/bindings/pyroot/ROOT.py"

In [ ]:
_root.CreateScopeProxy( "TTree" ).AsMatrix = _TTreeAsMatrix

In [28]:
%%bash
cd /home/blue/ROOT/root && git show 661a44e481255f24e1991fd164453d6cbfa66704

commit 661a44e481255f24e1991fd164453d6cbfa66704
Author: Stefan Wunsch <stefan.wunsch@cern.ch>
Date:   Tue Apr 17 15:03:39 2018 +0200

    [PyROOT] Add TTree.AsMatrix functionality

diff --git a/bindings/pyroot/ROOT.py b/bindings/pyroot/ROOT.py
index 92fe807e48..65b1eebba3 100644
--- a/bindings/pyroot/ROOT.py
+++ b/bindings/pyroot/ROOT.py
@@ -259,6 +259,110 @@ for pyclass in [
         class_scope._proxy__array_interface__ = _proxy__array_interface__
         class_scope.__array_interface__ = property(class_scope._proxy__array_interface__)
 
+# TTree.AsMatrix functionality
+def _TTreeAsMatrix(self, columns=None, exclude=None, dtype="double", return_labels=False):
+    # Import numpy lazily
+    try:
+        import numpy as np
+    except:
+        raise ImportError("Failed to import numpy during call of TTree.AsMatrix.")
+
+    # Check that tree has entries
+    if self.GetEntries() == 0:
+        raise Exception("Tree {} has no entries.".format(self.GetName()))
+
+    # Get all column

## Calling python from C++ from python

[demo1](demo1.ipynb)

[demo2](demo2.ipynb)

## Automatic creation of bindings from a cmake project

In [29]:
# TODO (couldn't make it work, contacted Wim about it)